In [3]:
import numpy as np
import torch
import torchvision
from torchvision.models.detection import maskrcnn_resnet50_fpn
import os
import cv2
import matplotlib.pyplot as plt
from os import listdir
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.utils import draw_segmentation_masks
import matplotlib.patches as patches

In [2]:
path = "../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant"
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


class breast_dataset(torch.utils.data.Dataset):

    def __init__(self,path,transforms=None):
        self.path = path
        self.transfrom = transforms
        self.img_name = []
        self.mask_name = []
        for img in listdir(self.path):
            if img[-8:] == "mask.png" :
                self.mask_name.append(img)
            elif img[-5:] == ").png":
                self.img_name.append(img)
    def __getitem__(self,idx):

        img_path = os.path.join(path,self.img_name[idx])
        mask_path = os.path.join(path,self.mask_name[idx])
        img = cv2.imread(img_path)
        img_gray = cv2.cvtColor(img,cv2.COLOR_BGR2RGB).astype(np.float32)
        img_nor = img_gray /255
        img_res = torch.as_tensor(img_nor).to(device).permute(2,0,1)
        mask = cv2.imread(mask_path)
        mask_gray = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
        a,mask_01 = cv2.threshold(mask_gray,127,255,cv2.THRESH_BINARY)
        contours,hierarchy = cv2.findContours(mask_01,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
        x,y,w,h = cv2.boundingRect((contours[0]))
        mask_ = mask_01.reshape(1,mask_01.shape[0],mask_01.shape[1])
        
        
        labels = [1]
        boxes = [x,y,x+w,y+h]
        target = {}
        target["masks"] = torch.as_tensor(mask_/255,dtype=torch.int64).to(device)
        target["labels"] = torch.as_tensor(labels, dtype=torch.int64).to(device)
        target["boxes"] = torch.as_tensor([boxes]).to(device)
        return img_res,target

    def __len__(self):
        return len(self.img_name)

In [ ]:
def model_mask_rcnn(num_class):
    model = maskrcnn_resnet50_fpn(pretrained=True)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features,num_class)
    return model


def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
breast = breast_dataset(path)
data_load = torch.utils.data.DataLoader(
    breast, batch_size=5, shuffle=True,num_workers=0, collate_fn=collate_fn)

num = 2
model = model_mask_rcnn(num)
model.to(device)
num_epoch = 35
param = [param for param in model.parameters() if param.requires_grad]
optimizer = torch.optim.SGD(param, lr=0.002, momentum=0.9)
lr_scheduler = None
for epoch in range(num_epoch):
    loss_num = 0
    model.train()
    for img, targets in data_load:
        loss_dic = model(img, targets)
        loss = sum(loss for loss in loss_dic.values())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_num += loss
    if lr_scheduler is not None:
        lr_scheduler.step()
    print("eppch:{},loss:{}".format(epoch, loss_num))


In [ ]:
cpu_device = torch.device("cpu")
model = model.eval()
test_path = '../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/malignant (1).png'
img_1 = cv2.imread(test_path)
img_2 = torch.as_tensor([img_1/255],dtype=torch.float32).permute(0,3,1,2).cuda()
outputs = model(img_2)


In [ ]:
def apply_nms(orig_prediction, iou_thresh=None):
    
    # torchvision returns the indices of the bboxes to keep
    keep = torchvision.ops.nms(orig_prediction['boxes'], orig_prediction['scores'], iou_thresh)
    
    final_prediction = orig_prediction
    final_prediction['boxes'] = final_prediction['boxes'][keep]
    final_prediction['scores'] = final_prediction['scores'][keep]
    final_prediction['labels'] = final_prediction['labels'][keep]
    
    return final_prediction

# function to convert a torchtensor back to PIL image
def torch_to_pil(img):
    return torchtrans.ToPILImage()(img).convert('RGB')


def plot_img_bbox(img, target):
    # plot the image and bboxes
    # Bounding boxes are defined as follows: x-min y-min width height
    fig, a = plt.subplots(1,1)
    fig.set_size_inches(5,5)
    a.imshow(img)
    for box in (target['boxes']):
        x, y, width, height  = box[0], box[1], box[2]-box[0], box[3]-box[1]
        rect = patches.Rectangle((x, y),
                                 width, height,
                                 linewidth = 2,
                                 edgecolor = 'r',
                                 facecolor = 'none')

        # Draw the bounding box on top of the image
        a.add_patch(rect)
    plt.show()

In [ ]:
mask = outputs[0]["masks"][0] > 0.5
img_1 = cv2.imread(test_path)
img_1 = cv2.cvtColor(img_1,cv2.COLOR_BGR2RGB)
img_2 = torch.as_tensor(img_1).permute(2,0,1)
dog_mask = draw_segmentation_masks(img_2,masks=mask,alpha=0.9)
out = np.array(dog_mask.permute(1,2,0))
plt.subplot(1,4,1)
plt.imshow(out)
plt.subplot(1,4,2)
plt.imshow(np.array(mask.detach().permute(1,2,0).to(torch.device("cpu"))).squeeze())
plt.subplot(1,4,3)
plt.imshow(cv2.imread("../input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/malignant/malignant (1)_mask.png"))
plt.subplot(1,4,4)
plt.imshow(img_1)
plt.show()

In [ ]:
from torchvision import transforms as torchtrans
nms_prediction = apply_nms(outputs[0], iou_thresh=0.1)
plot_img_bbox(torch_to_pil(img_2), nms_prediction)